# Source code thu thập dữ liệu

---

## Import

In [1]:
import pandas as pd
import os # thao tác với file, folder
import time # time.sleep()
import urllib.robotparser # kiểm tra file robots.txt
from requests_html import HTML # parse HTML
from selenium import webdriver # parse trang có Javascript
from datetime import datetime as dt # thao tác với ngày tháng
from datetime import timezone as tz # thao tác với ngày tháng

---

Danh sách các công ty sẽ được thu thập dữ liệu chứng khoán.

In [153]:
company_dict = {'AAPL': 'APPLE', 'AMZN': 'AMAZON', 'FB': 'FACEBOOK', 
                'GOOGL': 'GOOGLE', 'IBM': 'IBM', 'MSFT': 'MICROSOFT', 
                'NFLX': 'NETFLIX', 'TSLA': 'TESLA'}

## Thu thập dữ liệu bằng Parse HTML

- Thu thập từ trang: https://finance.yahoo.com/
- Dữ liệu thu thập là dữ liệu lịch sử chứng khoán theo từng ngày (daily, mỗi ngày một record) của các công ty.

In [2]:
def yahoofinance_html(symbol, start_date='01011999', end_date='26122020'):
    """
    - symbol: mã chứng khoán (ví dụ AMZN: Amazon, TSLA: Tesla)
    - start_date, end_date: ngày bắt đầu và kết thúc của dữ liệu giao dịch cần lấy
        - format: ddmmyyy, ví dụ 02092020 = ngày 2 tháng 9 năm 2020
    ----------
    Trả về pd.DataFrame dữ liệu chứng khoáng theo từng ngày của mã `symbol`
    """
    # Chuyển ngày sang Unix time (yahoofinance cần format này trong đường dẫn)
    start_date = str(int(dt.strptime(start_date, '%d%m%Y').replace(tzinfo=tz.utc).timestamp()))
    end_date   = str(int(dt.strptime(  end_date, '%d%m%Y').replace(tzinfo=tz.utc).timestamp()))
    
    url = 'https://finance.yahoo.com/quote/' + symbol + '/history?' + \
            'period1=' + start_date + '&period2=' + end_date + \
            '&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'
    
    # Kiểm tra file robots.txt xem có được phép crawl trang web
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url('https://finance.yahoo.com/robots.txt')
    rp.read()
    if not rp.can_fetch('*', url):
        assert("Không được phép tải " + url)
        
    driver.get(url)
        
    # Kéo xuống cho đến khi trang đã tải hết bảng dữ liệu
    while driver.page_source.find('Loading more data...</span>') >= 0:
        driver.execute_script("document.getElementsByTagName('table')[0].scrollIntoView(false)")
        time.sleep(0.5)
    
    # Rút trích table html để pandas parse thành DataFrame
    html = HTML(html=driver.page_source)
    df = pd.read_html(html.find('table', first=True).raw_html)[0]
    
    return df

Code chạy thực thi.

In [4]:
# Tải ở https://github.com/mozilla/geckodriver/releases
driver = webdriver.Firefox(executable_path='./geckodriver-v0.28.0-linux64/geckodriver')

Có thể tuỳ chỉnh thời gian muốn thu thập ở dòng (\*), thời gian mặc định là từ 01-01-1999 tới 26-12-2020

In [5]:
path = '../data/daily'
if not os.path.exists(path):
    os.makedirs(path)
for company in company_dict.keys():
    f = path + '/daily_' + company + '.csv'
    if os.path.exists(f):
        continue
    df = yahoofinance_html(symbol=company) # (*)
    df.to_csv(f, index=False)

In [6]:
driver.close()

## Thu thập dữ liệu bằng API

- Thu thập qua API của trang: https://www.alphavantage.co
- Dữ liệu thu thập là dữ liệu lịch sử chứng khoán với các giao dịch cách nhau một khoảng thời gian theo phút (gọi là intraday, mỗi ngày có nhiều record) của các công ty.

In [152]:
def alphavantage_api(function, symbol, apikey, outputsize='compact', interval=30):
    """
    - function: loại time series cần lấy
        - DAILY_ADJUSTED: các giao dịch theo từng ngày
        - INTRADAY_EXTENDED: các giao dịch trong ngày của từng ngày (tối đa 2 năm)
    - symbol: mã chứng khoán (ví dụ AMZN: Amazon, TSLA: Tesla)
    - apikey: mã để sử dụng api
    Tham số áp dụng cho function là DAILY_ADJUSTED:
        - outputsize: 
            + compact:lấy 100 dòng dữ liệu ngày gần đây nhất
            + full: lấy tối đa 20 năm dữ liệu hoặc 
                    từ ngày công ty mở giao dịch nếu công ty thành lập ít hơn 20 năm
    Tham số áp dụng cho function là INTRADAY_EXTENDED:
        - interval: thời gian (số phút) ngắn nhất giữa hai lượt giao dịch liên tục được lấy
            + Các giá trị được hỗ trợ: 1, 5, 15, 30, 60 (min)
    ----------
    Trả về pd.DataFrame
    """
    base_url = 'https://www.alphavantage.co/query?' + \
                'function=TIME_SERIES_' + function + '&symbol=' + symbol + '&apikey=' + apikey
    data = None
    
    if function == 'DAILY_ADJUSTED':
        url = base_url + '&datatype=csv' + '&outputsize=' + outputsize
        while(True):
            try: # Phòng trường hợp kết nối lỗi
                data = pd.read_csv(url)
                # Alphavantage giới hạn 5 truy cập api trên 1 phút
                # Nếu đạt giới hạn truy cập, api sẽ trả về thông báo, lúc này DataFrame chỉ có 2 dòng
                if len(data) > 2: break
            except: pass
            time.sleep(61)
    
    if function == 'INTRADAY_EXTENDED':
        url = base_url + '&interval=' + str(interval) + 'min' + '&slice='
        data = pd.DataFrame()
        for year, month in ((str(y), str(m)) for y in [1, 2] for m in range(1, 13)):
            # slice: dữ liệu được chia làm 24 phần tương ứng với 24 tháng của 2 năm
                # year1month1 là tháng gần nhất so với hiện tại
                # year2month12 là tháng xa nhất
            _slice = 'year' + year + 'month' + month
            while(True):
                try:
                    more_data = pd.read_csv(url + _slice)
                    if len(more_data) > 2: break
                except: pass
                time.sleep(61)
            data = data.append(more_data)
            time.sleep(3)
            
    return data

Code chạy thực thi.

In [ ]:
apikey = 'ZF5PTL6XOV5FHI9F'

for interval in [5, 30, 60]: # Thay đổi list này để lấy data với interval khác
    path = '..data/intraday_' + str(interval) + 'min'
    if not os.path.exists(path):
        os.mkdir(path)
    for company in company_dict.keys():
        f = path + '/intraday_' + company + '_' + str(interval) + 'min.csv'
        if os.path.exists(f):
            continue
        df = alphavantage_api(function='INTRADAY_EXTENDED', symbol=company, apikey=apikey, interval=interval)
        df.to_csv(f)

### Note
Dữ liệu của yahoo có close đã điều chỉnh cho tách cổ phiếu (split), còn adjusted_close điều chỉnh thêm chia cỗ tức (dividend).
> \*Close price adjusted for splits.**Adjusted close price adjusted for both dividends and splits.

Footnote của trang yahoofinance

### Bookmark

https://beenje.github.io/blog/posts/parsing-html-tables-in-python-with-pandas/

### Câu hỏi
- Giá cổ phiếu sẽ thay đổi như thế nào khi có split? Kì vọng giá sẽ tăng trong thời gian ngắn
- Volume thay đổi như thế nào theo thời gian? Xu hướng chung là tăng dần
- Covid ảnh hưởng? Kì vọng volume và close giảm 
- Chỉ số rủi ro ra sao? Kì vọng giá cổ phiếu càng cao thì rủi ro càng cao 
- Tương quan giua volume và price?

Overall, the adjusted closing price will give you a better idea of the overall value of the stock and help you make informed decisions about buying and selling, while the closing stock price will tell you the exact cash value of a share of stock at the end of the trading day.